## Задачи для совместного разбора

In [ ]:
import xlwings as xw
import xlsxwriter
import pandas as pd
import numpy as np
import csv
from IPython.display import Image

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
ws = xw.Book('себестоимостьА_в1.xlsx').sheets['Рецептура']
resipe = pd.DataFrame(ws.range('рцп_пшеничный_хлеб').value).fillna(value = np.nan)
cost_price = {resipe.loc[val][1] : np.round(sum([va * pr for va, pr in zip(resipe.loc[val][5:], resipe.loc[10][5:]) if va * pr >= 0]), 2) for val in range(3,7)}
res = list(cost_price.values())
cost_price = pd.DataFrame({'Name' : list(cost_price.keys()), 'Cost_price' : list(cost_price.values())})
cost_price

,Name,Cost_price
0,Хлеб Венгерский,21.48
1,"Хлеб ""Барбари""",16.52
2,Багет парижский,17.42
3,Чиабатта,18.08


2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
ws.range('P5').value = 'Себестоимость'
ws.range('P7').options(transpose = True).value = res

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
ws.range('P5').autofit()
ws.range('P5').font.bold = True
ws.range('P5').api.VerticalAlignment = 1

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
reviews = pd.read_csv('reviews_sample.csv', delimiter = ',', names = ['First', 'user_id', 'recipe_id', 'date', 'rating', 'review'])
recipes = pd.read_csv('recipes_sample.csv', delimiter = ',')
reviews.drop(0, axis = 0, inplace = True)
recipes = recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
recipes

,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,386977,zwetschgenkuchen bavarian plum cake,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,103312,zwiebelkuchen southwest german onion cake,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,486161,zydeco soup,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


In [ ]:
red = np.random.choice(len(recipes), int(len(recipes) * 0.05), replace = False)
red1 = np.random.choice(len(reviews), int(len(reviews) * 0.05), replace = False)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
xs = xw.Book('recipes.xlsx')
xs.sheets.add(name = 'Рецепты').range('A1').value = [list(recipes)] + [list(recipes.iloc[i].values) for i in red]
xs.sheets.add(name = 'Отзывы').range('A1').value = [list(reviews)] + [list(reviews.iloc[i].values) for i in red1]

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
xs = xw.sheets['Рецепты']
xs1 = xw.sheets['Отзывы']
xs.range('G1').options(transpose = True).value = ['seconds_assign'] + [i * 60 for i in xs.range('C2').expand('down').value]

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
xs.range('H1').value = 'seconds_formula'
xs.range('H2:H1501').formula = f'=$C2 * 60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
xs.range('B1:H1').autofit()
xs.range('A1:H1').font.bold = True
xs.range('A1:H1').api.HorizontalAlignment = -4131

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
minutes = xs.range('A1').expand().options(pd.DataFrame).value['minutes']
colors = [(255, 0, 0) if x > 10 else (0, 255, 0) if x < 5 else (255, 255, 0) for x in minutes]
for i in range(len(colors)):
    xs.range('C'+str(2 + i)).color = colors[i]

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
recipe = xs.range('A1').expand().options(pd.DataFrame).value
review = xs1.range('A1').expand().options(pd.DataFrame).value
k = list(recipe.merge(review, left_on = 'id', right_on = 'recipe_id')['recipe_id'])
xs.range('I1').options(transpose = True).value = ['n_reviews'] + [k.count(i) for i in list(recipe.index)]

In [ ]:
for i in range(2, 1502):
    xs.range('I' + str(i)).value = f'q=СЧЕТЕСЛИ(Отзывы!$C2:$C6335; A{i})'
    xs.range('I' + str(i)).characters[0].api.Delete()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def vilidate(xs, recipe, review):
    for i in range(1, len(review)):
        if (review.iloc[i]['rating'] >= 0 and review.iloc[i]['rating'] <=5 and list(recipe.index).count(review.iloc[i]['recipe_id']) !=0) == False:
            xs.range('A'+str(i)+':F'+str(i)).color = (255, 0, 0)
vilidate(xs1, recipe, review)